# Метод внешних штрафов

In [1]:
import numpy as np
import numpy.linalg as lg

## Функция и ее градиент

In [2]:
def fx(x):
    return x[0] ** 2 + x[1] ** 2 - 10 * x[0] - 8 * x[1]

def gradfx(x):
    arr = np.array([2 * x[0] - 10, 
                    2 * x[1] - 8, 
                    0 
                   ])
    return arr;

## Условие и его градиент

In [3]:
def Hx(x):
    return (max(0, -x[0])) ** 2 + (max(0, -x[1])) ** 2 + (max(0, -x[2])) ** 2 + (2 * x[0] + 3 * x[1] + x[2] - 6) ** 2

def gradHx(x):
    arr = np.array([- 2 * max(0, -x[0]) + 4 * (2 * x[0] + 3 * x[1] + x[2] - 6),
                    - 2 * max(0, -x[1]) + 6 * (2 * x[0] + 3 * x[1] + x[2] - 6),
                    - 2 * max(0, -x[2]) + 2 * (2 * x[0] + 3 * x[1] + x[2] - 6)
                    ])
    return arr;

## вспомогательные функции, которые не стоит использовать

In [56]:
def mygeom(x0, e, grad):
    ##print('    start mygeom')
    a = 0
    scalar = abs(sum(grad(x0)*grad(x0 - a * grad(x0))))
    while scalar > e: 
        a+=e
        scalar1 = scalar
        scalar = abs(sum(grad(x0)*grad(x0 - a * grad(x0))))
        #print('    a = ', a)#print('    scalar = ', scalar)
        if scalar1 < scalar:
            #print('    break', scalar)#print('        break', scalar) #print('        break', scalar1)
            break
    ##print('    a = ', a)
    ##print('    scalar = ', scalar)
    ##print('    end mygeom')
    return a
#------------------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------------------#
def func(f, x0, gradx0):
    def fun(a):
        A = x0 -  gradx0 * a
        return f(A)
    return fun

# первая производная
#------------------------------------------------------------------------------------------------------------#
def der1_e(f_, x0, a, gradx0, e):
    e = e / 2
    f_(x0)
    f = func(f_, x0, gradx0)
    def der(a):
        return (f(a + e) - f(a - e)) / (e * 2)
    return der
#------------------------------------------------------------------------------------------------------------#
# вторая производная
#------------------------------------------------------------------------------------------------------------#
def der2_e(f_, x0, a, gradx0, e):
    e = e / 2
    df = der1_e(f_, x0, a, gradx0, e)
    def der2(a):
        return (df(a + e) - df(a - e)) / ( e * 2 )
    return der2
#------------------------------------------------------------------------------------------------------------#
# ищем а при одномерной минимизации по направлению
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a1(f, x0, x1, e):
    a = 0
    df = der1_e(f, x0, a, x1-x0, e / 5)
    while e < abs(df(a)):
        d2f = der2_e(f, x0, a, x1-x0, e / 5)
        a = a - df(a)/d2f(a)
        df = der1_e(f, x0, a, x1-x0, e / 5)
    return a
#------------------------------------------------------------------------------------------------------------#
# тоже самое, только на вход подается начало вектора и сам вектор, а не его начало и конец
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a2(f, x0, vec, e):
    a = 0
    df = der1_e(f, x0, a, vec, e / 5)
    while e < abs(df(a)):
        d2f = der2_e(f, x0, a, vec, e / 5)
        a = a - df(a)/d2f(a)
        df = der1_e(f, x0, a, vec, e / 5)
    return a

## получение одномерной функции и золотое сечение

In [57]:
def func0(f, x0, gradx0):
    def fun(a):
        A = x0 -  gradx0 * a
        return f(A)
    return fun
#------------------------------------------------------------------------------------------------------------#
#золотое сечение
#------------------------------------------------------------------------------------------------------------#
def golden_section(f, a_, b_, e):
    n = 0
    a = a_
    b = b_
    x = a
    c = (3 - 5 ** 0.5 ) / 2 * (b - a) + a
    d = (5 ** 0.5 - 1) / 2 * (b - a) + a
    while (b - a) >= 2 * e:
        n+=1
        yc = f(c)
        yd = f(d)
        if(yc < yd):
            b = d
            d = c
            c = (3 - 5 ** 0.5 ) / 2 * (b - a) + a
        else:
            a = c
            c = d
            d = (5 ** 0.5 - 1) / 2 * (b - a) + a
    
    '''
    if (n%10 == 0) or (20 > n > 4) or ( n % 10 > 4):
        print('сделано ', n, 'шагов')
    elif(5 > n % 10 > 1):
        print('сделано ', n, 'шага')
    elif (n % 10 == 1):
        print('сделан ', n, 'шаг')
    '''
    return a

In [70]:
#------------------------------------------------------------------------------------------------------------#
# golden_section --- 
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a01(f_, x0, x1, e):
    f = func0(f_, x0, x1-x0)
    a = golden_section(f, -5., 5., e/10)
    return a
#------------------------------------------------------------------------------------------------------------#
# golden_section --- начало вектора и сам вектор
#------------------------------------------------------------------------------------------------------------#
def grad_descent_a02(f_, x0, vec, e):
    f = func0(f_, x0, vec)
    a = golden_section(f, -5., 5., e/10)
    return a

## МНГС

In [71]:
def min_gradient(f, grad, x0, e):
    n = 0
    while abs(lg.norm(grad(x0))) > e:
        n+=1
        print('')
        print('------[----', n, '----]------')
        #a = mygeom(x0, e/1000, grad)
        a = grad_descent_a02(f, x0, grad(x0), e/10)
        print('')
        print('a = ', a)
        print('  x  = ', x0)
        print('f(x) = ', f(x0))
        print(' |grad(x)| = ', lg.norm(grad(x0)))
        x0 = x0 - a * grad(x0)
    print('сделано ', n, ' шагов')
    return x0

### Ускоренный градиентный метод p-того порядка

In [72]:
#------------------------------------------------------------------------------------------------------------#
def p_gradient(f, grad, x0, e):
    n  = 0
    n1 = 0;
    while abs(lg.norm(grad(x0))) > e:
        x1 = x0
        
        for k in range(len(x0) * 4):
            if(abs(lg.norm(grad(x1))) <= e):
                x0 = x1
                break
            n+=1
            print('')
            print('------[----', n, '----]------')
            #a = mygeom(x1, e/1000, grad)
            a = grad_descent_a02(f, x1, grad(x1), e / 1000)
            x1 = x1 - grad(x1) * a

            print('')
            print('a = ', a)
            print('  x  = ', x1)
            print('f(x) = ', f(x1))
            if f(x1) >= f(x0):
                break
        if(abs(lg.norm(grad(x0))) <= e):
            break
        if f(x1) >= f(x0):
            break
        print('-------------------------------------------------------------------')
        print('----------------------------скачок[----', n1, '----]скачок')
        a = grad_descent_a1(f, x0, x1, e / 100)
        x0 = x0 -  (x1 - x0) * a
        print('----------------------------  a1  = ', a)
        print('----------------------------  x1  = ', x0)
        print('----------------------------f(x1) = ', f(x0))
        n1+=1
        
    print('сделано ', n, ' шагов')
    print('сделано ', n1, ' скачков')
    return x0
#------------------------------------------------------------------------------------------------------------#



#------------------------------------------------------------------------------------------------------------#
def ravine_gradient(f, grad, x0, e):
    x1 = x0 - e*10
    n  = 0
    n1 = 0
    while abs(lg.norm(grad(x0))) > e:
        for k in range(len(x0)):
            if(abs(lg.norm(grad(x0))) < e):
                break
            n+=1
            print('')
            print('------[----', n, '----]------')
            
            #a = mygeom(x0, e/100, grad)
            a = grad_descent_a02(f, x1, grad(x1), e / 10)
            x0 = x0 - grad(x0) * a
            
            #a = mygeom(x1, e/100, grad)
            a1 = grad_descent_a02(f, x1, grad(x1), e / 10)
            x1 = x1 - grad(x1) * a1
            
            print('  x0    = ', x0 ,   ' x1    = ', x1 )
            print('  f(x0) = ', f(x0), ' f(x1) = ', f(x1) )
            
        if(abs(lg.norm(grad(x0))) < e):
                break
                
        print('')
        print('скачок[----', n1, '----]скачок')
        a = grad_descent_a1(f, x0, x1, e / 100)
        x0 = x0 - a * (x1-x0)
        x1 = x0 - e*10
        print('a1 = ', a)
        print('  x1  = ', x0)
        print('f(x1) = ', f(x0))
        n1+=1
        
    print('сделано ', n, ' шагов')
    print('сделано ', n1, ' скачков')
    return x0
#------------------------------------------------------------------------------------------------------------#

## Реализаия метода

### Суммарная функция *f(x)+rH(x)*

In [73]:
def sumf(f, r , H):
    def result(x):
        res = f(x) + r * H(x)
        return res
    return result

In [78]:
def r_0(n):
    return np.sqrt(np.sqrt(n))
def r_1(n):
    return np.sqrt(n)
def r_2(n):
    return ((n * 2 + 10) ** (1/2) + 10)
def r_3(n):
    return ((n * 2 + 5 ) ** (1/2) + 5 )
def r_active(n):
    return ((n * 2 + 6 ) ** (1/2) + 6 )
def r_5(n):
    return ((n * 2 + 10) ** (1/2) + 10)

def sanction( f_, gradf_, H, gradH, x, e):
    n = 1
    r = r_active(n)
    f = sumf(f_, r, H)
    grad = sumf(gradf_, r, gradH)
    
    while( H(x) > e) or (lg.norm(grad(x)) > e):
        print('------------------------------------------------------------------------[----', n, '----]------')
        #x = min_gradient(f, grad, x, e)#лучше разделить на 10
        #x = p_gradient(f, grad, x, e)
        x = ravine_gradient(f, grad, x, e)
        n += 1
        r = r_active(n)
        f = sumf(f_, r, H)
        grad = sumf(gradf_, r, gradH)
        
        print(' -------------------------------------------------------------- f(x) + (( ', r ,' )) * H(x)    =    ', f(x))
        print(' --------------------------------------------------------------   x       = ', x)
        print(' -------------------------------------------------------------- H(x)      = ', H(x))
        print(' -------------------------------------------------------------- |grad(x)| = ', lg.norm(grad(x)))
        
          
    print('алгоритм был выполнен ( ', n - 1, ' ) раз')
    return x

## тесты

In [79]:
x0 = np.array([33/13, 4/13, 0.])
print('x0         = ', x0)
print('fx(x0)     = ', fx(x0))
print('H(x0)      = ', Hx(x0))
print('gradfx(x0) = ', gradfx(x0))
print('gradHx(x0) = ', gradHx(x0))

x0         =  [2.53846154 0.30769231 0.        ]
fx(x0)     =  -21.307692307692307
H(x0)      =  0.0
gradfx(x0) =  [-4.92307692 -7.38461538  0.        ]
gradHx(x0) =  [0. 0. 0.]


In [80]:
x0 = np.array([3., 3.7, 2.])
e = 0.05
x_min = sanction(fx, gradfx, Hx, gradHx, x0, e)
print(round(fx(x_min), 3), x_min)

------------------------------------------------------------------------[---- 1 ----]------

------[---- 1 ----]------
  x0    =  [1.31888074 1.1585264  1.15210899]  x1    =  [1.21089345 1.24471262 0.84628251]
  f(x0) =  -5.237873686921661  f(x1) =  -10.183619438693961

------[---- 2 ----]------
  x0    =  [1.1820572  0.9336415  1.07020339]  x1    =  [1.10893807 1.07031056 0.78141514]
  f(x0) =  -16.53220257768679  f(x1) =  -16.886396061849847

------[---- 3 ----]------
  x0    =  [1.17771596 0.89272322 1.04334587]  x1    =  [1.11125531 1.03619421 0.75741406]
  f(x0) =  -16.682703418935038  f(x1) =  -17.024362102536173

скачок[---- 0 ----]скачок
a1 =  -3.7671003086683483
  x1  =  [ 0.92735202  1.43319283 -0.03378793]
f(x1) =  -17.686781151469813

------[---- 4 ----]------
  x0    =  [ 0.94161222  1.4286142  -0.03939998]  x1    =  [ 0.83362493  1.51480042 -0.31286421]
  f(x0) =  -17.75532164104436  f(x1) =  -16.510500761815603

------[---- 5 ----]------
  x0    =  [ 0.9560259   1.421583

  x0    =  [ 2.56656502  0.3544366  -0.09896376]  x1    =  [ 2.48993987  0.48607542 -0.22061189]
  f(x0) =  -21.547440661088487  f(x1) =  -21.15411435849281
сделано  3  шагов
сделано  0  скачков
 -------------------------------------------------------------- f(x) + ((  12.6332495807108  )) * H(x)    =     -21.544497939328398
 --------------------------------------------------------------   x       =  [ 2.56656502  0.3544366  -0.09896376]
 -------------------------------------------------------------- H(x)      =  0.019295412277966134
 -------------------------------------------------------------- |grad(x)| =  0.11995466828841149
------------------------------------------------------------------------[---- 19 ----]------

------[---- 1 ----]------
  x0    =  [ 2.56642701  0.35420808 -0.09887567]  x1    =  [ 2.42405007  0.39809018 -0.39162929]
  f(x0) =  -21.544521852669615  f(x1) =  -17.912098297792642

------[---- 2 ----]------
  x0    =  [ 2.56638944  0.35412801 -0.09872657]  x1    = 

In [81]:
x0 = np.array([33/13, 4/13, 0])
e = 0.05
x_min = sanction(fx, gradfx, Hx, gradHx, x0, e)
print(round(fx(x_min), 3), x_min)

------------------------------------------------------------------------[---- 1 ----]------

------[---- 1 ----]------
  x0    =  [2.556508 0.334762 0.      ]  x1    =  [ 2.44852071  0.43339524 -0.27346424]
  f(x0) =  -21.4739002140845  f(x1) =  -21.05775879725333

------[---- 2 ----]------
  x0    =  [ 2.55923753  0.33885629 -0.00759231]  x1    =  [ 2.47709538  0.47434658 -0.25082999]
  f(x0) =  -21.494788183021097  f(x1) =  -21.50822114118912

------[---- 3 ----]------
  x0    =  [ 2.56383175  0.34574763 -0.03312952]  x1    =  [ 2.48411987  0.47863992 -0.22431906]
  f(x0) =  -21.54852719474015  f(x1) =  -21.54059822444652

скачок[---- 0 ----]скачок
a1 =  -0.48920121387221704
  x1  =  [ 2.5248366   0.4107587  -0.12665967]
f(x1) =  -21.636385110373745

------[---- 4 ----]------
  x0    =  [ 2.52288086  0.40691964 -0.12851273]  x1    =  [ 2.41489357  0.49888196 -0.40197697]
  f(x0) =  -21.639700372773955  f(x1) =  -20.582468776666737

------[---- 5 ----]------
  x0    =  [ 2.52320952  0

------[---- 2 ----]------
  x0    =  [ 2.56430519  0.35030339 -0.09080921]  x1    =  [ 2.48224162  0.48445398 -0.33337313]
  f(x0) =  -21.525575249231522  f(x1) =  -20.67604988017627

------[---- 3 ----]------
  x0    =  [ 2.56420732  0.35003716 -0.08991364]  x1    =  [ 2.48831183  0.48545078 -0.22711787]
  f(x0) =  -21.525620843004884  f(x1) =  -21.04779395175787
сделано  3  шагов
сделано  0  скачков
 -------------------------------------------------------------- f(x) + ((  13.874007874011811  )) * H(x)    =     -21.52358029773448
 --------------------------------------------------------------   x       =  [ 2.56420732  0.35003716 -0.08991364]
 -------------------------------------------------------------- H(x)      =  0.015936632612673758
 -------------------------------------------------------------- |grad(x)| =  0.09634474025237104
------------------------------------------------------------------------[---- 28 ----]------

------[---- 1 ----]------
  x0    =  [ 2.56409939  0.34985